## 한글 NLP 
* 네이버 영화 평점 데이터

In [8]:
import konlpy
import pandas as pd

train_df=pd.read_csv('ratings_train.txt', sep='\t', encoding ='utf-8')
train_df[:10]
test_df=pd.read_csv('ratings_test.txt', sep='\t', encoding ='utf-8')
train_df['label'].value_counts() #0부정 1긍정
test_df['label'].value_counts() #0부정 1긍정


1    25173
0    24827
Name: label, dtype: int64

### 데이터 전처리 

In [9]:
import re
# na를 공백으로
train_df=train_df.fillna('')
test_df=test_df.fillna('')

#정규표현식 : 숫자를 공백으로 변경
train_df['document']=train_df['document'].apply(lambda x: re.sub(r'\d+', '', x))
test_df['document']=test_df['document'].apply(lambda x: re.sub(r'\d+', '', x))

#id 칼럼 삭제
train_df.drop('id', axis=1, inplace=True)
test_df.drop('id', axis=1, inplace=True)

In [11]:
#10%만 랜덤하게 뽑기
train_df=train_df.sample(frac=0.1)
test_df=test_df.sample(frac=0.1)

* 형태소 단어로 토큰화
* morphs() 메서도 이용-토큰화해 객체 리스트로 반환 

In [15]:
from konlpy.tag import Okt
okt= Okt()
def tw_tokenizer(text):
    tokens_ko=twitter.morphs(text)
    return tokens_ko

## 피처 벡터화  TF-IDF 

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

tfidf_vect= TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3,
                           max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train=tfidf_vect.transform(train_df['document'])

C:\Users\user\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


* 로지스틱

In [25]:
lg_clf= LogisticRegression(random_state=0, solver='liblinear')

#파라미터 c최적화를 위해 gridsearch cv 이용
params={'C':[1,3.5,4.5,5.5,10]}
grid_cv=GridSearchCV(lg_clf,param_grid=params, cv=3, scoring='accuracy',
                    verbose=1)
grid_cv.fit(tfidf_matrix_train, train_df['label'])
print(grid_cv.best_params_, round(grid_cv.best_score_, 4))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
{'C': 3.5} 0.8099


* testdf 이용해 최종 감성 분석 예측
* 테스트 세트 이용해 예측할경우 학습할때 적용한 Tfidvectirozer 그대로 사용
* 학습시 설정된 TfidVectorizer의 피처개수와 테스트데이터로 변환할 개수가 같아짐

In [26]:
from sklearn.metrics import accuracy_score
#학습 데이터를 적용한 tfid이용해 test df를 TFIDF 값으로 피처 변환
tfidf_matrix_test=tfidf_vect.transform(test_df['document'])
#gridsearch에서 최적파라미터로 학습된 classifier 그대로 이용 
best_estimator=grid_cv.best_estimator_
preds= best_estimator.predict(tfidf_matrix_test)

print('Logistic Regression accuracy:', accuracy_score(test_df['label'],preds))

Logistic Regression accuracy: 0.8214
